**版本**: v1.3.12 (修复Gradio界面调试信息显示和清理不准确描述) | **更新时间**: 2025-08-24

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zhurong2020/claude-colab-projects/blob/main/demos/medical-ocr/medical-ocr-demo.ipynb) [![GitHub](https://img.shields.io/badge/GitHub-源代码-blue?logo=github)](https://github.com/zhurong2020/claude-colab-projects/tree/main/demos/medical-ocr)

## 🎯 功能特性
- 📄 支持医疗文档图像文字识别
- 🤖 使用PaddleOCR高精度识别引擎
- 📊 自动生成结构化CSV报告
- 🖼️ 支持多种图像格式输入
- 💡 简单易用的交互界面
- 🏠 完整本地开发环境支持
- 🌐 支持中英文混合识别
- 🏗️ 独立应用架构，可直接在Colab运行

## 🔧 更新内容 (v1.3.12)
- **🔧 Gradio修复**: 修复web界面调试信息无法显示的问题
- **📊 调试捕获**: 使用redirect_stdout捕获OCR处理过程的调试信息
- **🎯 问题定位**: 调试信息现在会显示在web界面中，便于问题诊断
- **📝 文档清理**: 清理代码中不准确的"修复XXX问题"描述
- **🔄 版本追踪**: 按约定更新最小版本号便于观察变化

## 🚀 使用说明

### Colab环境
1. 点击上方的"Open in Colab"按钮
2. 运行环境检查和依赖安装
3. 上传医疗文档图像
4. 执行OCR文字识别
5. 下载CSV结果文件

### 本地环境
```bash
# 从项目根目录一键启动
../start_local.sh

# 手动启动
source ../venv/bin/activate && jupyter notebook
```

### 目录结构
```
medical-ocr/
├── medical-ocr-demo.ipynb    # 本演示文件
├── gradio_demo.py           # Web界面版本
├── test_chinese_encoding_fix.py  # 中文编码测试
└── assets/                  # 资源文件
    ├── sample_docs/         # 示例文档
    └── results/            # OCR结果
```

---
*使用 Claude Code 开发，支持 Google Colab 和本地运行 🚀*

In [ ]:
# ================================
# 环境检查和基础设置
# ================================

import warnings
warnings.filterwarnings("ignore")

def check_environment():
    """检查运行环境并显示系统信息"""
    print("🔍 检查运行环境...")
    
    # 检查是否在Colab环境
    try:
        import google.colab # type: ignore # noqa: F401 # 需要用于环境检测
        print("✅ 运行在Google Colab")
        in_colab = True
    except ImportError:
        print("ℹ️ 运行在本地环境")
        in_colab = False
    
    # 检查GPU
    try:
        import torch
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"✅ 计算设备: {device}")
        if device == 'cuda':
            print(f"✅ GPU型号: {torch.cuda.get_device_name(0)}")
            print(f"✅ GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    except ImportError:
        print("ℹ️ PyTorch未安装，使用CPU模式")
    
    return in_colab

# 运行环境检查
in_colab = check_environment()

In [ ]:
# ================================
# 安装必要的依赖包
# ================================

def install_dependencies():
    """安装项目所需的依赖包"""
    print("📦 安装医疗OCR项目依赖...")
    
    import subprocess
    import sys
    
    # 核心依赖包列表
    packages = [
        'paddlepaddle',
        'paddleocr',
        'pandas',
        'pillow',
        'opencv-python',
        'tqdm',
        'gradio'
    ]
    
    for package in packages:
        try:
            if package == 'opencv-python':
                import cv2  # type: ignore # noqa: F401 # 用于验证opencv安装
                print(f"✅ {package} 已安装")
            elif package == 'pillow':
                from PIL import Image  # type: ignore # noqa: F401 # 用于验证PIL安装
                print(f"✅ {package} 已安装")
            else:
                __import__(package.replace('-', '_'))
                print(f"✅ {package} 已安装")
        except ImportError:
            print(f"📥 安装 {package}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    
    print("✅ 所有依赖安装完成!")

# 安装依赖
install_dependencies()

In [ ]:
# ================================
# 导入必要的库
# ================================

try:
    import pandas as pd  # type: ignore
    from tqdm import tqdm
    from paddleocr import PaddleOCR  # type: ignore
    import gradio as gr  # type: ignore
    print("📚 所有库导入成功!")
except ImportError as e:
    print(f"❌ 库导入失败: {e}")
    print("💡 请先运行依赖安装单元格")

In [ ]:
# ================================
# 医疗OCR核心功能类
# ================================

class MedicalOCRProcessor:
    def __init__(self):
        """初始化医疗OCR处理器"""
        print("🏥 初始化医疗OCR处理器...")
        
        # 检查GPU可用性
        try:
            import torch
            use_gpu = torch.cuda.is_available()
            gpu_info = f"GPU可用: {use_gpu}"
            if use_gpu:
                gpu_info += f" (设备: {torch.cuda.get_device_name(0)})"
            print(f"⚡ {gpu_info}")
        except ImportError:
            use_gpu = False
            print("ℹ️ PyTorch未安装，使用CPU模式")
        
        # 初始化PaddleOCR，使用兼容的配置
        try:
            # 使用兼容的参数初始化PaddleOCR (v3.1.1)
            self.ocr = PaddleOCR(use_angle_cls=True, lang='ch')
            print("✅ 使用兼容参数初始化OCR引擎")
        except Exception as e:
            print(f"❌ OCR初始化失败: {e}")
            self.ocr = None
            raise RuntimeError(f"PaddleOCR初始化失败: {e}")
        
        print("✅ OCR引擎初始化完成")
    
    def _preprocess_image(self, image_path):
        """预处理图像，确保格式和质量适合OCR"""
        try:
            from PIL import Image as PILImage
            import os # type: ignore
            
            # 打开并验证图像
            with PILImage.open(image_path) as img:
                print(f"📊 原始图像信息: 尺寸={img.size}, 模式={img.mode}")
                
                # 转换为RGB格式（如果不是的话）
                if img.mode != 'RGB':
                    print(f"🔄 转换图像模式: {img.mode} -> RGB")
                    img = img.convert('RGB')
                
                # 检查图像尺寸，如果过大则适当缩小
                max_size = 2048
                if max(img.size) > max_size:
                    print(f"🔄 调整图像尺寸: {img.size}")
                    ratio = max_size / max(img.size)
                    new_size = (int(img.size[0] * ratio), int(img.size[1] * ratio))
                    img = img.resize(new_size, PILImage.Resampling.LANCZOS)
                    print(f"✅ 新尺寸: {img.size}")
                
                # 保存预处理后的图像
                processed_path = image_path.replace('.png', '_processed.png').replace('.jpg', '_processed.jpg').replace('.jpeg', '_processed.jpg')
                if processed_path == image_path:
                    processed_path = image_path.replace('.', '_processed.')
                
                img.save(processed_path, quality=95, optimize=False)
                print(f"💾 预处理图像已保存: {processed_path}")
                
                return processed_path
                
        except Exception as e:
            print(f"⚠️ 图像预处理失败: {e}，使用原始图像")
            return image_path

    def _parse_ocr_result(self, result):
        """解析OCR结果 - 兼容多种PaddleOCR返回格式"""
        extracted_texts = []
        try:
            if not result or not isinstance(result, list) or not result[0]:
                print("⚠️ OCR结果为空或格式不正确")
                return []

            page_result = result[0]

            # 新版 PaddleOCR v3.1.1+ (返回带 .json 属性的 OCRResult 对象)
            if hasattr(page_result, 'json'):
                print("✅ 检测到PaddleOCR v3.1.1+ OCRResult对象格式")
                json_result = page_result.json
                if isinstance(json_result, dict) and 'res' in json_result:
                    res_data = json_result['res']
                    if res_data and 'rec_texts' in res_data and 'rec_scores' in res_data:
                        texts, scores = res_data['rec_texts'], res_data['rec_scores']
                        print(f"📊 识别到文本数量: {len(texts) if texts else 0}")
                        if texts and scores:
                            for text, score in zip(texts, scores):
                                if text and text.strip():
                                    extracted_texts.append({'text': text.strip(), 'confidence': float(score)})
                return extracted_texts

            # 兼容直接返回字典列表的格式
            elif isinstance(page_result, list) and page_result and isinstance(page_result[0], dict) and 'text' in page_result[0]:
                print("✅ 检测到字典列表格式")
                for line in page_result:
                    text, confidence = line.get('text', ''), line.get('confidence', 0.0)
                    if text.strip():
                        extracted_texts.append({'text': text.strip(), 'confidence': float(confidence)})
                return extracted_texts

            # 兼容旧版 PaddleOCR (返回包含元组的列表)
            elif isinstance(page_result, list):
                print("✅ 检测到传统列表格式")
                for line_result in page_result:
                    if (line_result and len(line_result) >= 2 and
                        line_result[1] and len(line_result[1]) >= 2):
                        text, confidence = line_result[1]
                        if text and text.strip():
                            extracted_texts.append({'text': text.strip(), 'confidence': float(confidence)})
                return extracted_texts
            
            else:
                print(f"⚠️ 未知的OCR结果格式: {type(page_result)}")
                self._debug_result_structure(result)

        except Exception as e:
            print(f"⚠️ 结果解析失败: {e}")
            import traceback
            print(f"详细错误: {traceback.format_exc()}")
        
        return extracted_texts

    def extract_text_from_image(self, image_path):
        """从图像中提取文字 - 增强版本"""
        import os
        
        if self.ocr is None:
            print("❌ OCR引擎未初始化")
            return []
        
        try:
            # 验证图像文件
            if not os.path.exists(image_path):
                print(f"❌ 图像文件不存在: {image_path}")
                return []
            
            if os.path.getsize(image_path) == 0:
                print(f"❌ 图像文件为空: {image_path}")
                return []
            
            print(f"📄 正在处理图像: {image_path}")
            print(f"📊 文件大小: {os.path.getsize(image_path)} 字节")
            
            # 预处理图像：确保图像格式和质量适合OCR
            processed_image_path = self._preprocess_image(image_path)
            
            # 使用PaddleOCR进行识别
            result = None
            extracted_texts = []
            
            # 使用predict方法 (推荐的新版本API)
            try:
                print("🔄 尝试使用predict方法...")
                result = self.ocr.predict(processed_image_path)
                print(f"✅ predict方法调用成功，结果类型: {type(result)}")
                extracted_texts = self._parse_ocr_result(result)
                
                if extracted_texts:
                    print(f"✅ 成功识别 {len(extracted_texts)} 行文字")
                    return extracted_texts
                
            except Exception as e1:
                print(f"⚠️ predict方法失败: {e1}")
                
                # 尝试使用传统的ocr方法
                try:
                    print("🔄 尝试使用传统ocr方法...")
                    result = self.ocr.ocr(processed_image_path)  # type: ignore
                    print(f"✅ OCR方法调用成功，结果类型: {type(result)}")
                    extracted_texts = self._parse_ocr_result(result)
                    
                    if extracted_texts:
                        print(f"✅ 成功识别 {len(extracted_texts)} 行文字")
                        return extracted_texts
                        
                except Exception as e2:
                    print(f"❌ 所有可用的OCR调用方法都失败")
                    print(f"详细错误: predict={e1}, ocr={e2}")
            
            # 如果所有方法都没有识别到文字
            if not extracted_texts:
                print("⚠️ 未检测到任何文字内容")
                self._debug_result_structure(result)
                self._check_image_quality(processed_image_path)
                
                return []
            
            return extracted_texts
        
        except Exception as e:
            print(f"❌ 图像处理失败: {str(e)}")
            import traceback
            print(f"详细错误信息: {traceback.format_exc()}")
            return []
    
    def _debug_result_structure(self, result):
        """调试结果结构"""
        try:
            print(f"🔍 调试信息: result类型={type(result)}")
            if result:
                print(f"🔍 result长度: {len(result) if hasattr(result, '__len__') else 'N/A'}")
                if isinstance(result, list) and len(result) > 0:
                    first_item = result[0]
                    print(f"🔍 第一项类型: {type(first_item)}")
                    if isinstance(first_item, dict):
                        print(f"🔍 字典键: {list(first_item.keys())}")
                    elif hasattr(first_item, '__dict__'):
                        print(f"🔍 对象属性: {list(vars(first_item).keys())}")
                    elif isinstance(first_item, list) and len(first_item) > 0:
                        print(f"🔍 嵌套列表长度: {len(first_item)}")
                        if len(first_item) > 0:
                            print(f"🔍 嵌套项类型: {type(first_item[0])}")
        except Exception as e:
            print(f"🔍 调试信息获取失败: {e}")
    
    def _check_image_quality(self, image_path):
        """检查图像质量"""
        try:
            from PIL import Image as PILImage
            import os
            
            if not os.path.exists(image_path):
                print("🔍 图像文件不存在")
                return
            
            with PILImage.open(image_path) as img:
                width, height = img.size
                total_pixels = width * height
                
                print(f"🔍 图像质量检查:")
                print(f"   尺寸: {width}x{height} ({total_pixels:,} 像素)")
                print(f"   格式: {img.format}")
                print(f"   模式: {img.mode}")
                
                # 质量评估
                if total_pixels < 50000:
                    print("   ⚠️ 图像分辨率较低，可能影响识别效果")
                elif total_pixels > 4000000:
                    print("   ℹ️ 图像分辨率很高，处理速度可能较慢")
                else:
                    print("   ✅ 图像分辨率适中")
                
        except Exception as e:
            print(f"🔍 图像质量检查失败: {e}")
    
    def process_single_image(self, image_path):
        """处理单个图像文件"""
        import os
        print(f"📄 处理图像: {os.path.basename(image_path)}")
        
        # 提取文字
        extracted_texts = self.extract_text_from_image(image_path)
        
        # 整理结果
        results = []
        for i, item in enumerate(extracted_texts):
            results.append({
                'file_name': os.path.basename(image_path),
                'line_number': i + 1,
                'extracted_text': item['text'],
                'confidence': round(item['confidence'], 4)
            })
        
        return results
    
    def process_multiple_images(self, image_paths):
        """批量处理多个图像文件"""
        all_results = []
        
        print(f"📊 开始批量处理 {len(image_paths)} 个图像文件...")
        
        for image_path in tqdm(image_paths, desc="处理进度"):
            results = self.process_single_image(image_path)
            all_results.extend(results)
        
        return all_results
    
    def save_results_to_csv(self, results, output_path):
        """保存结果到CSV文件"""
        if not results:
            # 如果没有结果，创建空的DataFrame
            df = pd.DataFrame(columns=['file_name', 'line_number', 'extracted_text', 'confidence'])
        else:
            df = pd.DataFrame(results)
        
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"💾 结果已保存到: {output_path}")
        return df

# 初始化OCR处理器
print("🔧 正在初始化OCR处理器...")
try:
    ocr_processor = MedicalOCRProcessor()
    print("✅ OCR处理器初始化成功!")
except Exception as e:
    print(f"❌ OCR处理器初始化失败: {e}")
    print("💡 请检查PaddleOCR安装是否正确")

In [ ]:
# 创建示例医疗文档（用于演示）- 优化字体加载和显示 v1.3.12

In [ ]:
# ================================
# 演示OCR功能
# ================================

def demo_ocr_functionality():
    """演示OCR功能"""
    print("🚀 开始演示医疗OCR功能...")
    
    # 确保使用正确的图像路径
    import os
    sample_image_path = sample_doc # type: ignore
    if not os.path.exists(sample_image_path):
        # 尝试assets目录中的示例文档
        assets_path = 'assets/sample_docs/sample_medical_document.png'
        if os.path.exists(assets_path):
            sample_image_path = assets_path
        else:
            sample_image_path = 'sample_medical_document.png'
    
    print(f"📄 使用图像文件: {sample_image_path}")
    
    # 处理示例文档
    results = ocr_processor.process_single_image(sample_image_path)
    
    # 显示识别结果
    print("\n📊 文字识别结果:")
    print("-" * 60)
    
    for result in results:
        print(f"行{result['line_number']:2d}: {result['extracted_text']} (置信度: {result['confidence']:.3f})")
    
    # 保存结果到CSV (assets/results目录)
    os.makedirs('assets/results', exist_ok=True)
    csv_path = 'assets/results/ocr_results_demo.csv'
    df = ocr_processor.save_results_to_csv(results, csv_path)
    
    print(f"\n📈 共识别出 {len(results)} 行文字")
    print(f"📄 结果已保存到 CSV 文件: {csv_path}")
    
    # 显示CSV内容预览
    if len(results) > 0:
        print("\n📋 CSV文件预览:")
        print(df.to_string(index=False))
    else:
        print("\n⚠️ 没有识别到文字内容，请检查图像文件")
    
    return df, csv_path

# 运行演示
demo_df, demo_csv = demo_ocr_functionality()

In [ ]:
# ================================
# 图像上传处理模块
# ================================

import os # type: ignore
import numpy as np  # type: ignore # noqa: F401
from PIL import Image as PILImage  # type: ignore # noqa: F401

print("🔍 图像处理模块已加载，准备接收上传图像...")

In [ ]:
# ================================
# Gradio Web交互界面
# ================================

def create_gradio_interface():
    """创建Gradio Web界面用于图像上传和OCR处理"""
    
    def process_uploaded_image(image):
        """处理上传的图像"""
        import tempfile
        import os
        
        if image is None:
            return "❌ 请上传一个图像文件", None
        
        try:
            print(f"📸 收到上传图像，类型: {type(image)}")
            print(f"📸 图像尺寸: {image.size if hasattr(image, 'size') else 'N/A'}")
            print(f"📸 图像模式: {image.mode if hasattr(image, 'mode') else 'N/A'}")
            
            # 创建临时文件保存上传的图像
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp_file:
                temp_image_path = tmp_file.name
                print(f"💾 创建临时文件: {temp_image_path}")
            
            # 确保图像是RGB格式并保存
            if hasattr(image, 'mode') and image.mode != 'RGB':
                print(f"🔄 转换图像模式: {image.mode} -> RGB")
                image = image.convert('RGB')
            
            # 保存图像
            image.save(temp_image_path, 'PNG', quality=95)
            print(f"💾 图像已保存到临时文件")
            
            # 验证临时文件
            if not os.path.exists(temp_image_path):
                return "❌ 临时文件创建失败", None
                
            file_size = os.path.getsize(temp_image_path)
            print(f"📊 临时文件大小: {file_size} 字节")
            
            if file_size == 0:
                return "❌ 图像文件为空，保存失败", None
            
            print("🚀 开始OCR处理...")
            
            # 使用OCR处理器处理图像
            results = ocr_processor.process_single_image(temp_image_path)
            
            print(f"📊 OCR处理完成，识别到 {len(results) if results else 0} 行文字")
            
            # 清理临时文件
            try:
                os.unlink(temp_image_path)
                print("🗑️ 临时文件已清理")
            except Exception as e:
                print(f"⚠️ 临时文件清理失败: {e}")
            
            # 格式化输出结果
            if results and len(results) > 0:
                output_text = f"🎉 成功识别出 {len(results)} 行文字:\\n\\n"
                for result in results:
                    output_text += f"行{result['line_number']:2d}: {result['extracted_text']} (置信度: {result['confidence']:.3f})\\n"
                
                # 创建CSV数据用于下载
                import pandas as pd
                import io
                
                df = pd.DataFrame(results)
                csv_buffer = io.StringIO()
                df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')
                csv_data = csv_buffer.getvalue()
                
                # 将CSV数据写入临时文件以供下载
                with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv', encoding='utf-8-sig') as csv_file:
                    csv_file.write(csv_data)
                    csv_file_path = csv_file.name
                
                print(f"📄 CSV文件已创建: {csv_file_path}")
                return output_text, csv_file_path
            else:
                # 提供更详细的调试信息
                debug_info = "🔍 调试信息:\\n"
                debug_info += f"- 图像尺寸: {image.size}\\n" if hasattr(image, 'size') else ""
                debug_info += f"- 图像模式: {image.mode}\\n" if hasattr(image, 'mode') else ""
                debug_info += f"- 临时文件大小: {file_size} 字节\\n"
                debug_info += "\\n可能的原因:\\n"
                debug_info += "1. 图像中没有清晰的文字\\n"
                debug_info += "2. 图像分辨率过低或过高\\n"
                debug_info += "3. 文字颜色与背景对比度不足\\n"
                debug_info += "4. OCR引擎初始化问题\\n"
                debug_info += "\\n建议:\\n"
                debug_info += "- 确保图像清晰且文字可读\\n"
                debug_info += "- 尝试使用高对比度的图像\\n"
                debug_info += "- 检查图像是否包含中文或英文文字"
                
                return f"😞 未检测到任何文字内容\\n\\n{debug_info}", None
                
        except Exception as e:
            error_msg = f"❌ 图像处理失败: {str(e)}"
            print(f"ERROR: {error_msg}")
            import traceback
            print(f"详细错误: {traceback.format_exc()}")
            return f"{error_msg}\\n\\n🔍 详细错误信息已输出到控制台，请检查", None
    
    # 创建Gradio界面
    with gr.Blocks(title="医疗文档OCR识别", theme=gr.themes.Soft()) as demo: # type: ignore
        gr.Markdown("""
        # 🏥 医疗文档OCR识别系统
        
        上传医疗文档图像，系统将自动识别其中的文字内容并生成CSV报告
        
        **支持格式**: PNG, JPG, JPEG  
        **识别语言**: 中文、英文  
        **输出格式**: CSV文件
        """)
        
        with gr.Row():
            with gr.Column():
                # 图像上传组件
                image_input = gr.Image(
                    label="📄 上传医疗文档图像",
                    type="pil",
                    height=400
                )
                
                # 处理按钮
                process_btn = gr.Button("🚀 开始识别", variant="primary", size="lg")
                
            with gr.Column():
                # 结果显示
                result_text = gr.Textbox(
                    label="📊 识别结果",
                    lines=15,
                    max_lines=20,
                    placeholder="识别结果将在这里显示...",
                    show_copy_button=True
                )
                
                # CSV下载
                csv_download = gr.File(
                    label="💾 下载CSV结果",
                    visible=False
                )
        
        # 示例图片展示
        gr.Markdown("### 📋 使用示例")
        with gr.Row():
            # 检查示例文件是否存在
            example_path = "assets/sample_docs/sample_medical_document.png"
            if os.path.exists(example_path):
                gr.Examples(
                    examples=[[example_path]],
                    inputs=image_input,
                    label="点击加载示例医疗文档"
                )
            else:
                gr.Markdown("⚠️ 示例文档未找到，请直接上传您的医疗文档图像")
        
        # 使用说明
        gr.Markdown("""
        ### 💡 使用说明
        1. **上传图像**: 点击上方区域上传医疗文档图像
        2. **开始识别**: 点击"开始识别"按钮进行OCR处理
        3. **查看结果**: 识别结果将显示在右侧文本框中
        4. **下载CSV**: 处理完成后可下载CSV格式的结构化结果
        
        ### 🔧 技术特性
        - 🤖 基于PaddleOCR高精度识别引擎
        - 🌐 支持中英文混合识别
        - 📊 提供置信度评估
        - 💾 自动生成CSV格式结果
        - 🔍 支持多种图像格式
        
        ### ⚠️ 注意事项
        - 请确保图像清晰可读
        - 文字与背景对比度要足够
        - 图像分辨率建议在 500x500 到 2000x2000 像素之间
        - 文字识别结果仅供参考，重要医疗信息请人工核验
        
        ### 🐛 故障排除
        - 如果识别失败，请检查控制台输出的详细错误信息
        - 尝试使用不同格式或更清晰的图像
        - 确保图像中包含清晰可读的文字内容
        """)
        
        # 绑定处理函数
        def update_result(image):
            """更新结果并控制下载按钮可见性"""
            result_text, csv_path = process_uploaded_image(image)
            
            if csv_path:
                # 有CSV文件时显示下载组件
                return result_text, gr.File(value=csv_path, visible=True)
            else:
                # 无CSV文件时隐藏下载组件
                return result_text, gr.File(visible=False)
        
        process_btn.click(
            fn=update_result,
            inputs=image_input,
            outputs=[result_text, csv_download]
        )
    
    return demo

# 创建并启动界面
print("🌐 创建Gradio Web界面...")
try:
    demo_interface = create_gradio_interface()
    print("✅ Gradio界面创建成功!")
    print("💡 在本地环境中，可以使用 demo_interface.launch() 启动Web服务")
    print("💡 在Colab环境中，可以使用 demo_interface.launch(share=True) 获取公共链接")
    
    # 根据环境选择启动方式
    try:
        import google.colab  # type: ignore # noqa: F401 # 检测Colab环境
        print("🚀 检测到Colab环境，启动公共分享链接...")
        demo_interface.launch(share=True, height=800)
    except ImportError:
        print("🏠 本地环境检测成功")
        print("📝 如需启动Web界面，请运行: demo_interface.launch()")
        # 在本地环境中不自动启动，避免占用端口
        
except Exception as e:
    print(f"❌ Gradio界面创建失败: {e}")
    print("💡 请检查Gradio是否正确安装")

In [ ]:
# ================================
# OCR功能验证测试
# ================================

def test_ocr_functionality():
    """快速测试OCR功能是否正常工作"""
    print("🧪 开始OCR功能验证测试...")
    
    # 检查OCR处理器是否可用 - 修复全局变量检测和IDE警告
    try:
        # 使用globals()检查变量存在，避免IDE unbound警告
        if 'ocr_processor' not in globals():
            print("❌ 全局OCR处理器变量未定义")
            print("💡 请先运行'医疗OCR核心功能类'单元格来初始化OCR处理器")
            return False
        
        processor = globals()['ocr_processor']  # type: ignore # 动态访问全局变量
        print("✅ 找到全局OCR处理器变量")
        
    except Exception as e:
        print(f"❌ 访问OCR处理器时出错: {e}")
        return False
    
    # 检查处理器对象是否有效
    if processor is None:
        print("❌ OCR处理器对象为None")
        print("💡 OCR处理器初始化可能失败，请检查错误信息")
        return False
    
    # 检查PaddleOCR引擎
    if not hasattr(processor, 'ocr') or processor.ocr is None:
        print("❌ PaddleOCR引擎未初始化")
        print("💡 PaddleOCR初始化失败，请检查:")
        print("   1. 网络连接是否正常")
        print("   2. PaddleOCR是否正确安装")
        print("   3. 模型文件是否下载完成")
        return False
    
    print("✅ OCR处理器和引擎检查通过")
    
    # 检查示例文件是否存在
    import os
    sample_path = "assets/sample_docs/sample_medical_document.png"
    
    if not os.path.exists(sample_path):
        print(f"⚠️ 示例文件不存在: {sample_path}")
        
        # 尝试创建示例文件
        try:
            print("🎨 尝试创建示例文档...")
            # 检查create_sample_medical_document函数是否存在
            if 'create_sample_medical_document' not in globals():
                print("❌ create_sample_medical_document函数未定义")
                print("💡 请先运行'创建示例医疗文档'单元格")
                return False
            
            create_func = globals()['create_sample_medical_document']  # type: ignore # 动态访问全局函数
            sample_path = create_func()
            print(f"✅ 示例文档已创建: {sample_path}")
            
        except Exception as e:
            print(f"❌ 示例文档创建失败: {e}")
            return False
    else:
        print(f"✅ 找到示例文件: {sample_path}")
    
    # 测试OCR处理
    try:
        print(f"🔍 测试OCR处理: {sample_path}")
        results = processor.process_single_image(sample_path)
        
        print(f"📊 OCR处理完成，返回结果类型: {type(results)}")
        print(f"📊 结果数量: {len(results) if results else 0}")
        
        if results and len(results) > 0:
            print(f"✅ OCR测试成功！识别到 {len(results)} 行文字")
            print("📝 前3行识别结果:")
            for i, result in enumerate(results[:3]):
                print(f"   {i+1}. {result['extracted_text']} (置信度: {result['confidence']:.3f})")
            return True
        else:
            print("❌ OCR测试失败：未识别到文字")
            print("🔍 可能原因：")
            print("   1. PaddleOCR版本兼容性问题")
            print("   2. 模型文件下载不完整")
            print("   3. 示例图像质量问题")
            print("   4. API调用参数不兼容")
            
            # 尝试直接调用PaddleOCR
            print("\\n🔧 尝试直接调用PaddleOCR引擎...")
            try:
                direct_result = processor.ocr.predict(sample_path)
                print(f"🔍 直接调用结果类型: {type(direct_result)}")
                print(f"🔍 直接调用结果长度: {len(direct_result) if direct_result else 0}")
                
                if direct_result:
                    print("✅ PaddleOCR引擎本身工作正常")
                    print("💡 问题可能在结果解析逻辑中")
                else:
                    print("❌ PaddleOCR引擎调用也失败")
                    
            except Exception as direct_e:
                print(f"❌ 直接调用PaddleOCR失败: {direct_e}")
            
            return False
            
    except Exception as e:
        print(f"❌ OCR测试异常: {e}")
        import traceback
        print(f"详细错误: {traceback.format_exc()}")
        return False

# 运行OCR功能验证
print("🚀 运行OCR功能验证测试...")
ocr_test_result = test_ocr_functionality()

if ocr_test_result:
    print("\\n🎉 OCR功能验证成功！Gradio界面应该能正常工作")
    print("💡 现在可以安全使用Gradio界面进行图像上传和识别")
else:
    print("\\n⚠️ OCR功能验证失败！需要检查PaddleOCR配置")
    print("💡 建议按顺序执行以下步骤：")
    print("   1. 确认已运行'医疗OCR核心功能类'单元格")
    print("   2. 确认已运行'创建示例医疗文档'单元格")
    print("   3. 检查网络连接和模型下载状态")
    print("   4. 如果问题持续，请重启运行时环境")

# 显示当前可用的关键全局变量
print("\\n🔍 当前可用的关键全局变量:")
available_vars = []

# 安全检查各个关键变量的存在性
key_variables = {
    'ocr_processor': '医疗OCR处理器',
    'create_sample_medical_document': '示例文档创建函数', 
    'demo_interface': 'Gradio Web界面对象'
}

for var_name, description in key_variables.items():
    try:
        if var_name in globals():
            var_value = globals()[var_name]
            if var_value is not None:
                available_vars.append(f"✅ {var_name} ({description})")
            else:
                available_vars.append(f"❌ {var_name} (已定义但值为None)")
        else:
            available_vars.append(f"❌ {var_name} (未定义)")
    except Exception as e:
        available_vars.append(f"❌ {var_name} (访问错误: {e})")

for var_info in available_vars:
    print(f"   {var_info}")

# 如果demo_interface未定义，给出创建提示
if 'demo_interface' not in globals() or globals()['demo_interface'] is None:
    print("\\n💡 Gradio界面创建提示:")
    print("   如需创建Web界面，请运行'Gradio Web交互界面'单元格")
    print("   该单元格会创建demo_interface变量并初始化Web界面")

In [ ]:
# ================================
# 项目总结和使用指南
# ================================

def show_project_summary():
    """显示项目总结和使用指南"""
    
    summary = """ # type: ignore
🎉 医疗文档OCR识别演示项目运行完成!

📋 项目功能总结:
✅ 医疗文档图像文字识别
✅ 中英文混合文字识别
✅ 批量图像处理支持
✅ CSV格式结果导出
✅ 置信度评估
✅ 用户友好的交互界面

🛠️ 技术特性:
• 基于PaddleOCR高精度识别引擎
• 支持GPU加速处理
• 自动角度检测和矫正
• 结构化数据输出

📖 使用场景:
• 医疗处方识别
• 病历文档数字化
• 检查报告提取
• 医疗档案管理

🔗 项目地址:
GitHub: https://github.com/zhurong2020/mcr
Colab: 当前notebook链接

👨‍⚕️ 适用用户:
• 医疗工作者
• 数据分析师
• 研究人员
• 开发者

⚠️ 使用说明:
1. 确保图像清晰可读
2. 支持PNG、JPG等常见格式
3. 批量处理时注意文件大小
4. 结果仅供参考，重要信息请人工核验

💡 技术支持:
如有问题或建议，欢迎反馈交流！
    """
    
    print(summary)

# 显示项目总结
show_project_summary()